In [1]:
import os
import sys
sys.path.append('D:\\Users\\Hegxiten\\workspace\\Rutgers_Railway_security_research\\OOD_Train')

from datetime import datetime, timedelta
import numpy as np
import random
from train import Train
from infrastructure import Track, Block, BigBlock
from signaling import AutoSignal, HomeSignal, AutoPoint, ControlPoint

import networkx as nx


In [2]:
_node = {  0:ControlPoint(idx=0, ports=[0,1], MP=0.0), \
                    1:AutoPoint(1), \
                    2:AutoPoint(2), \
                    3:ControlPoint(idx=3, ports=[0,1,3], ban_routes={1:[3],3:[1]}), \
                    4:ControlPoint(idx=4, ports=[0,2,1], ban_routes={0:[2],2:[0]}), \
                    5:AutoPoint(5), \
                    6:ControlPoint(idx=6, ports=[0,1,3], ban_routes={1:[3],3:[1]}), \
                    7:ControlPoint(idx=7, ports=[0,2,1], ban_routes={0:[2],2:[0]}), \
                    8:AutoPoint(8), \
                    9:AutoPoint(9), \
                    10:ControlPoint(idx=10, ports=[0,1])}       
nbunch = [_node[i] for i in range(len(_node))]

_track = [  Track(nbunch[0], 1, nbunch[1], 0), Track(nbunch[1], 1, nbunch[2], 0), Track(nbunch[2], 1, nbunch[3], 0),\
            Track(nbunch[3], 1, nbunch[4], 0), Track(nbunch[3], 3, nbunch[4], 2, edge_key=1),\
            Track(nbunch[4], 1, nbunch[5], 0), Track(nbunch[5], 1, nbunch[6], 0),\
            Track(nbunch[6], 1, nbunch[7], 0), Track(nbunch[6], 3, nbunch[7], 2, edge_key=1),\
            Track(nbunch[7], 1, nbunch[8], 0), Track(nbunch[8], 1, nbunch[9], 0), Track(nbunch[9], 1, nbunch[10], 0)]
ebunch = [_track[i] for i in range(len(_track))]

# _node and _track will be parameters passed from outside in the future development
G = nx.MultiGraph()
for n in nbunch:
    G.add_node(n, attr=n.__dict__, instance=n)              
    # __dict__ of instances (CPs, ATs, Tracks) is pointing the same 
    # attribute dictionary as the node in the MultiGraph

for t in ebunch:
    G.add_edge(t.L_point, t.R_point, key=t.edge_key, attr=t.__dict__, instance=t)          
    # __dict__ of instances (CPs, ATs, Tracks) is pointing the same 
    # attribute dictionary as the edge in the MultiGraph
    # key is the index of parallel edges between two nodes
    t.L_point.port_track[t.entry_port_L] = t.R_point.port_track[t.entry_port_R] = t

for i in G.nodes():     # register the neighbor nodes as observers to each node
    i.neighbors.extend([n for n in G.neighbors(i)])              
    for n in G.neighbors(i):
        i.add_observer(n)

In [6]:
F = G.copy()        
# F is a shallow copy of G: attrbutes of G/F components 
# are pointing at the same memory.

def _get_new_edge(node, length=False):
    at_neighbor = [j for j in F.neighbors(i)]
    assert len(at_neighbor) == len(F.edges(i)) == 2
    edgetrk_L_points = [F[at_neighbor[0]][node][0]['instance'].L_point, F[node][at_neighbor[1]][0]['instance'].L_point]
    edgetrk_R_points = [F[at_neighbor[0]][node][0]['instance'].R_point, F[node][at_neighbor[1]][0]['instance'].R_point]
    edgetrk_L_points.remove(i)
    edgetrk_R_points.remove(i)
    new_edge_length = F[at_neighbor[0]][i][0]['instance'].length + F[i][at_neighbor[1]][0]['instance'].length
    if length:
        return edgetrk_L_points[0], edgetrk_R_points[0], new_edge_length
    else:  
        return edgetrk_L_points[0], edgetrk_R_points[0]

for i in G.nodes():
    # only use G.nodes() instead of F.nodes() to get original nodes 
    # to avoid dictionary size changing issues. 
    # all the following graph updates are targeted on F
    if i.type == 'at':
        new_L_point, new_R_point, new_length = _get_new_edge(i, length=True)
        assert len(F[new_L_point][i]) == len(F[i][new_R_point]) == 1
        new_track =  Track( new_L_point, F[new_L_point][i][0]['instance'].entry_port_L,\
                            new_R_point, F[i][new_R_point][0]['instance'].entry_port_R,\
                            edge_key=0, length=new_length)

        F.remove_node(i)
        F.add_edge(new_L_point, new_R_point, attr=new_track.__dict__, instance=new_track)     
        # MultiGraph parallel edges are auto-keyed (0, 1, 2...)
        # default 0 as mainline, idx as track number

for (u, v, k) in F.edges(keys=True):
    blk_path = nx.shortest_path(G, u, v)
    big_block_edges = [(blk_path[i], blk_path[i+1]) for i in range(len(blk_path) - 1)]
    big_block_instance = BigBlock(  u, F[u][v][k]['instance'].entry_port_L,\
                                    v, F[u][v][k]['instance'].entry_port_R,\
                                    edge_key=k, length=F[u][v][k]['instance'].length, \
                                    raw_graph=G, cp_graph=F)
    for (n, m) in big_block_edges:
        if G[n][m][k]['instance'] not in big_block_instance.tracks:
            big_block_instance.tracks.append(G[n][m][k]['instance'])
        # get the list of track unit components of a bigblock, and record in the instance

    F[u][v][k]['attr'] = big_block_instance.__dict__
    F[u][v][k]['instance'] = big_block_instance
    for t in F[u][v][k]['instance'].tracks:
        t.bigblock = F[u][v][k]['instance']

In [7]:
F[_node[0]][_node[3]][0]['instance'].traffic_direction = 'east'

In [10]:
G[_node[0]][_node[1]][0]['instance'].traffic_direction = 'east'